# Main

## Define variables and import the main library

In [ ]:
project_directory = '/data/BIDS-HPC/private/projects/dmi2'
import os, sys
links_dir = os.path.join(project_directory, 'data', 'all_gene_expression_files_in_target', 'links')
annotation_file = os.path.join(project_directory, 'data', 'gencode.v22.annotation.gtf')
sample_sheet_file = os.path.join(project_directory, 'data', 'gdc_sample_sheet.2020-07-02.tsv')
metadata_file = os.path.join(project_directory, 'data', 'metadata.cart.2020-07-02.json')
if os.path.join(project_directory, 'checkout') not in sys.path:
    sys.path.append(os.path.join(project_directory, 'checkout'))
import target_class_lib as tc

## Load the data downloaded from the GDC Data Portal

In [ ]:
df_samples, df_counts = tc.load_gdc_data(sample_sheet_file, metadata_file, links_dir)

## Calculate the FPKM and FPKM-UQ dataframes, and check them with known values if the needed datafiles are present

In [ ]:
df_fpkm, df_fpkm_uq = tc.get_fpkm(df_counts, annotation_file, df_samples, links_dir)

## Remove from the samples and intensities the samples that correspond to multiple cases (i.e., people)

In [ ]:
df_samples, indexes_to_keep, _ = tc.drop_multiperson_samples(df_samples)
df_counts = df_counts.iloc[indexes_to_keep,:]
df_fpkm = df_fpkm.iloc[indexes_to_keep,:]
df_fpkm_uq = df_fpkm_uq.iloc[indexes_to_keep,:]

## Perform exploratory data analysis on the sample labels

In [ ]:
tc.eda_labels(df_samples)

## Plot histograms of the numerical columns of the samples/labels before and after cutoffs are applied, and print out a summary of what was removed

In [ ]:
df_samples, indexes_to_keep = tc.remove_bad_samples(df_samples)
df_counts = df_counts.iloc[indexes_to_keep,:]
df_fpkm = df_fpkm.iloc[indexes_to_keep,:]
df_fpkm_uq = df_fpkm_uq.iloc[indexes_to_keep,:]

## Perform exploratory data analysis on the filtered sample labels

In [ ]:
tc.eda_labels(df_samples)

## Print some random data for us to spot-check in the files themselves to manually ensure we have a handle on the data arrays

In [ ]:
tc.spot_check_data(df_samples, df_counts, df_fpkm, df_fpkm_uq)

## Calculate the TPM using the counts and gene lengths
Note I've confirmed sufficient equality using TPM calculation using FPKM and FPKM-UQ

In [ ]:
df_tpm = tc.get_tpm(df_counts, annotation_file)

## Add a labels column based on the project id and sample type columns and show the unique values by decreasing frequency

In [ ]:
df_samples['label 1'] = df_samples['project id'] + ', ' + df_samples['sample type']
print(df_samples['label 1'].value_counts())

## Run the variance-stabilizing transformation using DESeq2 using this most-detailed set of labels

In [ ]:
X1, y1 = tc.run_vst(df_counts, df_samples['label 1'], project_directory)

## Create and plot PCA and tSNE analyses using the variance-stabilizing-transformed data from DESeq2

In [ ]:
# Perform PCA
import sklearn.decomposition as sk_decomp
pca = sk_decomp.PCA(n_components=10)
pca_res = pca.fit_transform(X1.iloc[:,:500])
#pca_res = pca.fit_transform(X1.iloc[:,:])
print('Top {} PCA explained variance ratios: {}'.format(10, pca.explained_variance_ratio_))
ax = tc.plot_unsupervised_analysis(pca_res, y1)
ax.set_title('PCA - variance-stabilizing transformation')

# Perform tSNE analysis
import sklearn.manifold as sk_manif
tsne = sk_manif.TSNE(n_components=2)
tsne_res = tsne.fit_transform(X1.iloc[:,:500])
#tsne_res = tsne.fit_transform(X1.iloc[:,:])
ax = tc.plot_unsupervised_analysis(tsne_res, y1)
ax.set_title('tSNE - variance-stabilizing transformation')

From the above tSNE plot, we find many interesting things, e.g.:

* ALL is spread out in multiple places (all oranges/yellows and early greens), into four main clusters; in particular:
  * ALL/P2/PBDC (both /BM and /PB) is located in three of them
  * ALL/P2/RBDC (i.e., /BM) is basically isolated to one of them
  * ALL/P3 is basically in the fourth cluster
* AML has its own single cluster
* There are some ALL in an AML cluster and vice versa, though the two corresponding clusters are near each other
* CCSK is very tightly clustered
* NBL has its own cluster, with one RT instance that may be misclassified
* OS is tightly clustered, except for two samples in WT, which may indicate that they are misclassified
* Aside from the one possibly misclassified RT, they are all solidly clustered
* WT is clustered together
* The two normal species are very tightly clustered together

## Based on the above observations, make a new set of more sensible labels

See comments in each line in the following code block

In [ ]:
df_samples['label 2'] = df_samples['label 1'] # initialize the new label (2) to the original, most-detailed label (1)
df_samples['label 2'][df_samples['label 2'].str.contains('normal', case=False, regex=False)] = 'TARGET-Normal' # set to "TARGET-Normal" any label including "normal" in its original name
df_samples['label 2'] = df_samples['label 2'].str.split(pat=', ', expand=True)[0] # otherwise, just set the label to the project ID (see definition of 'label 1' above)
df_samples['label 2'] = df_samples['label 2'].str.split(pat='-', expand=True)[1] # finally, at the same time, both remove "TARGET-" from the labels and, for ALL, remove "-PX", where X={1,2,3}
print(df_samples['label 2'].value_counts()) # print the final labels counts

## Re-label the unsupervised plots above, just as a quick check of our new labeling scheme

In [ ]:
# Define the new labels
y = df_samples['label 2'].copy()
y.index = y.index.str.replace('-', '_')
y = y.loc[y1.index] # order them the same way as the PCA/tSNE results we already calculated

# Redo the plots, with appropriate sorting so that the labels are in alphabetical order
sorting_indexes = y.to_numpy().argsort(axis=0)
ax = tc.plot_unsupervised_analysis(pca_res[sorting_indexes,:], y.iloc[sorting_indexes])
ax.set_title('PCA - variance-stabilizing transformation - with new labels')
ax = tc.plot_unsupervised_analysis(tsne_res[sorting_indexes,:], y.iloc[sorting_indexes])
ax.set_title('tSNE - variance-stabilizing transformation - with new labels')

Note that had we not previously plotted the most detailed set of labels, we would have been left forever wondering why the two distinct clusters of AML data.

## Now actually run the variance-stabilizing transformation using DESeq2 on the re-labeled dataset

In [ ]:
X2, y2 = tc.run_vst(df_counts, df_samples['label 2'], project_directory)

## Run PCA and tSNE on the new dataset

In [ ]:
# Perform PCA
import sklearn.decomposition as sk_decomp
pca = sk_decomp.PCA(n_components=10)
pca_res = pca.fit_transform(X2.iloc[:,:500])
print('Top {} PCA explained variance ratios: {}'.format(10, pca.explained_variance_ratio_))
ax = tc.plot_unsupervised_analysis(pca_res, y2)
ax.set_title('PCA - variance-stabilizing transformation')

# Perform tSNE analysis
import sklearn.manifold as sk_manif
tsne = sk_manif.TSNE(n_components=2)
tsne_res = tsne.fit_transform(X2.iloc[:,:500])
ax = tc.plot_unsupervised_analysis(tsne_res, y2)
ax.set_title('tSNE - variance-stabilizing transformation')

## Create a figure helping to explore the extent of sampling each unique label in the dataset (i.e., each group)

In [ ]:
fig = tc.explore_sample_size(X2, y2, tsne_res, n_range=range(100,601,100))
fig.savefig(os.path.join(project_directory, 'data', 'exploring_sample_size_100_601_100.png'), facecolor='w', dpi=150)

## Run some random forest classification models on the data, saving the results and plotting the accuracies of the models on the entire input datasets

In [ ]:
# Get a reasonable set of sample sizes
possible_n = [x for x in range(1,13)] + [x for x in range(15,46,5*2)] + [x for x in range(50,101,10*2)]

# Determine the samples of just particular types
nbl_or_normal_samples = (y2=='NBL').to_numpy() | (y2=='Normal').to_numpy() # get samples that are NBL or Normal
aml_or_normal_samples = (y2=='AML').to_numpy() | (y2=='Normal').to_numpy() # get samples that are AML or Normal

# Generate the random forest classification models
accuracies, possible_n, rnd_clf_holder = tc.generate_random_forest_models(X2, y2, project_directory, 'all_classes', ntrials=30, possible_n=possible_n) # all data
accuracies_nbl, possible_n_nbl, rnd_clf_holder_nbl = tc.generate_random_forest_models(X2.iloc[nbl_or_normal_samples,:], y2.iloc[nbl_or_normal_samples], project_directory, 'nbl_vs_normal', possible_n=possible_n[-10:], ntrials=30) # just NBL/Normal
accuracies_aml, possible_n_aml, rnd_clf_holder_aml = tc.generate_random_forest_models(X2.iloc[aml_or_normal_samples,:], y2.iloc[aml_or_normal_samples], project_directory, 'aml_vs_normal', possible_n=possible_n[-10:], ntrials=30) # just AML/Normal

# Plot the accuracy vs. sample size for each study type
tc.plot_accuracy_vs_sample_size(accuracies, possible_n, 'all classes')
tc.plot_accuracy_vs_sample_size(accuracies_nbl, possible_n_nbl, 'NBL vs. normal')
tc.plot_accuracy_vs_sample_size(accuracies_aml, possible_n_aml, 'AML vs. normal')

## Determine the genes in decreasing order of average feature importance

In [ ]:
important_genes, _ = tc.calculate_average_feature_importance(X2.columns, rnd_clf_holder)
important_genes_nbl, _ = tc.calculate_average_feature_importance(X2.columns, rnd_clf_holder_nbl)
important_genes_aml, _ = tc.calculate_average_feature_importance(X2.columns, rnd_clf_holder_aml)

## Print *ROUGHLY* top top-10 differentiating genes in each study

In [110]:
print('\nAll data\n', important_genes[:20])
print('\nNBL vs. normal\n', important_genes_nbl[:20])
print('\nAML vs. normal\n', important_genes_aml[:20])


All data
 ENSG00000164853    0.000953
ENSG00000132970    0.000903
ENSG00000137745    0.000896
ENSG00000112559    0.000874
ENSG00000257093    0.000806
ENSG00000139656    0.000805
ENSG00000226519    0.000776
ENSG00000099956    0.000766
ENSG00000150551    0.000752
ENSG00000250208    0.000746
ENSG00000232835    0.000745
ENSG00000125872    0.000734
ENSG00000075891    0.000709
ENSG00000108821    0.000708
ENSG00000130477    0.000707
ENSG00000198363    0.000702
ENSG00000111432    0.000676
ENSG00000157227    0.000661
ENSG00000235180    0.000640
ENSG00000219159    0.000625
Name: score, dtype: float64

NBL vs. normal
 ENSG00000144455    0.000533
ENSG00000175707    0.000533
ENSG00000130962    0.000533
ENSG00000109132    0.000533
ENSG00000132563    0.000500
ENSG00000166986    0.000500
ENSG00000168259    0.000500
ENSG00000227081    0.000500
ENSG00000102471    0.000500
ENSG00000105289    0.000500
ENSG00000172270    0.000500
ENSG00000243547    0.000500
ENSG00000244245    0.000500
ENSG00000261502    0

# Scratch